In [1]:
import pdfplumber
from llama_index.core import SimpleDirectoryReader, Document, VectorStoreIndex, StorageContext,  ChatPromptTemplate
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter, SentenceWindowNodeParser, SemanticSplitterNodeParser
# from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import Settings

from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)


from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.huggingface import HuggingFaceLLM, HuggingFaceInferenceAPI
from llama_index.llms.openai import OpenAI
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core import PromptTemplate



# from llama_index.core.postprocessor import LLMRerank, SentenceTransformerRerank
# from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.response.notebook_utils import display_source_node


import openai
from pinecone import Pinecone
import pinecone
import os
from dotenv import load_dotenv

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python311\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Python311\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Python311\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
 

c:\Python311\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


resource module not available on Windows


In [2]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
load_dotenv()

True

In [4]:

pc = Pinecone(api_key = os.environ["PINECONE_API_KEY"])

In [5]:
openai.api_key = os.environ["OPEN_AI_KEY"]


In [6]:
embbeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

c:\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [275]:
# embbeding_model._embed("Tôi là Tài")

## Processing, Chunking and storing data

In [38]:
reader = SimpleDirectoryReader(input_dir=r"C:\Users\leduc\OneDrive\Desktop\NLP\LLM-Agent\AI _solution\sample-data")
documents = reader.load_data(num_workers=4)

In [39]:
documents[0].get_content

<bound method TextNode.get_content of Document(id_='7c85ef8a-6cf1-4124-be73-ebaf4fcac1fb', embedding=None, metadata={'file_path': 'C:\\Users\\leduc\\OneDrive\\Desktop\\NLP\\LLM-Agent\\AI _solution\\sample-data\\text_0.txt', 'file_name': 'text_0.txt', 'file_type': 'text/plain', 'file_size': 598, 'creation_date': '2024-12-14', 'last_modified_date': '2024-08-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Môn học Kiến trúc hệ thống và Tích hợp hệ thống (IE302) giới thiệu các khái niệm về thu thập yêu cầu, tìm nguồn cung ứng, đánh giá và tích hợp các thành phần vào một hệ thống thống nhất. Nội dung môn học bao gồm các nguyên tắc quản lý dự án, sự liên kết giữa các ứng dụng CNTT và quy trình t

In [7]:
# llm = HuggingFaceLLM(model_name="vinai/phobert-base")
llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2", token=os.environ["HF_TOKEN"])

C:\Users\leduc\AppData\Local\Temp\ipykernel_10160\3330286725.py:2: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2", token=os.environ["HF_TOKEN"])


In [8]:
Settings.llm = llm
Settings.embed_model = embbeding_model

In [109]:
FAQ_prompt = """Ensure the questions are well-structured, grammatically correct, and contextually accurate.
You are a Vietnamese language expert and question-generation specialist. Based on the following context, generate exactly three concise, thoughtful, and relevant questions in Vietnamese. 

### Context:
{context_str}

### Guidelines:
1. The questions should focus on extracting meaningful insights or prompting further discussion about the context.
2. Only based on information about course information do not care about files information
3. Use natural and conversational Vietnamese suitable for the intended audience.
4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
5. remove all special token
6. Generate {num_questions}

### Example Output:
If the context is about environmental protection:
1. Bạn nghĩ gì về vai trò của giáo dục trong việc bảo vệ môi trường?
2. Những giải pháp nào có thể giúp giảm thiểu rác thải nhựa trong cuộc sống hàng ngày?
3. Làm thế nào cộng đồng có thể hợp tác để cải thiện chất lượng không khí?

### Output:



"""

In [110]:
keyword_prompt = {
  "context": "You are a Vietnamese language expert and Vietnamese keywords identifier specialist.",
  "task": "Generate {keywords} unique keywords for this document.",
  "document": "{context_str}",
  "guidelines": [
    "Only based on information about course information do not care about files information.",
    "MUST NOT return any explanation.",
    "Answer in Vietnamese.",
    # "Remove all special characters like brackets or code",
    # "Answer must be consistent"
  ],
  "examples":[
    "CS115, Toán, Khoa Học Máy Tính, 2023",
    "CE231, Tin học, Phần mềm",
    "SE112, Robotic, Vi mạch"  
  ],
  "output_format": "keywords: "
}
import json

keyword_prompt = json.dumps(keyword_prompt, ensure_ascii=False, indent=4)

In [111]:
splitter = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=80, embed_model=embbeding_model
)


In [112]:
transformations = [
    splitter,
    KeywordExtractor(keywords=1, llm=llm, prompt_template= keyword_prompt),
    QuestionsAnsweredExtractor(questions=1, llm=llm, prompt_template= FAQ_prompt)
]

In [113]:
pipeline = IngestionPipeline(transformations=transformations)

In [114]:
nodes = pipeline.run(documents=documents)

100%|██████████| 13/13 [00:14<00:00,  1.15s/it]


In [115]:
import re

def clean_text(text):

    text = re.sub(r'\d+\.\s*', '', text)  
    

    
    text = re.sub(r'-', ' ', text)
    text = re.sub(r'"', ' ', text)
    text = text.replace('\n', ' ').strip()
    text = re.sub(r'\s+', ' ', text) 
    text = re.sub(r'\bkeywords: ', '', text)
    text = text.strip()
    
    return text

In [118]:
nodes[0].metadata

{'file_path': 'C:\\Users\\leduc\\OneDrive\\Desktop\\NLP\\LLM-Agent\\AI _solution\\sample-data\\text_0.txt',
 'file_name': 'text_0.txt',
 'file_type': 'text/plain',
 'file_size': 598,
 'creation_date': '2024-12-14',
 'last_modified_date': '2024-08-06',
 'excerpt_keywords': 'keywords: Kiến trúc hệ thống, Tích hợp hệ thống, IE302, Quản lý dự án, Quy trình tổ chức, Ứng dụng CNTT, Chất lượng, Tìm nguồn cung ứng.',
 'questions_this_excerpt_can_answer': '1. Bạn học môn nào đó gọi là Kiến trúc hệ thống và Tích hợp hệ thống (IE302)?\n2. Trong môn học này, sinh viên sẽ được học đến gì về quản lý dự án?\n3. Tại sao ứng dụng CNTT liên kết với quy trình tổ chức trong môn học này?'}

In [116]:
for i in range(0, len(nodes)):
    # print(nodes[i].metadata["excerpt_keywords"])
    print(clean_text(nodes[i].metadata["excerpt_keywords"]))

Kiến trúc hệ thống, Tích hợp hệ thống, IE302, Quản lý dự án, Quy trình tổ chức, Ứng dụng CNTT, Chất lượng, Tìm nguồn cung ứng.
Toán, CS115, Khoa học máy tính, Toán ứng dụng, Máy học, Trí tuệ nhân tạo, Khai phá dữ liệu, Xử lý tín hiệu số.
Môn Thực tập NT115, doanh nghiệp, kinh nghiệm
Ngành Công nghệ Thông tin, Áp dụng từ Khoá 18, 2023, Chính quy tập trung, Số tín chỉ đào tạo, Tối thiểu 132 tín chỉ, Ngoại ngữ.
Thời gian đào tạo, 04 năm, 8 học kỳ chính.
Môn học NT505, Khóa luận tốt nghiệp, An ninh Thông tin, quy trình vận hành, hệ thống triển khai, đánh giá, phát triển tiếp theo.
kết quả nghiên cứu, đối tượng sinh viên, đề tài khóa luận, nhu cầu thực tế, giảng viên, chuyên sâu, khả năng nghiên cứu, làm việc độc lập.
CE407, Hệ thống nhúng và Robot, Chuyên ngành, Sinh viên, Học chuyên đề tốt nghiệp, Kiến thức, Tìm hiểu, Nghiên cứu, Giải quyết bài toán, Kỹ năng thiết kế, Kỹ thuật máy tính, Thuyết trình, Giao tiếp, Làm việc nhóm.
Môn học, hệ thống nhúng.
Phát triển và vận hành game (SE327)
Ch

In [92]:
import re
course_code_pattern = r"\b[A-Z]{2}[0-9]{3}\b"

In [93]:
doc = "Môn học CS221 về Xử lý ngôn ngữ tự nhiên SE221 nhằm cung cấp cho sinh IE690 viên những kiến thức cơ bản về chuyên ngành này, bao gồm văn phạm phi ngữ cảnh CFG, văn phạm DCG, cách cài đặt và giải thích cơ chế xử lý văn phạm DCG trên Prolog, và FSA."

re.findall(course_code_pattern, doc)

['CS221', 'SE221', 'IE690']

In [94]:
for i in range(0,len(nodes)):
    # print(re.findall(course_code_pattern, nodes[i].text))
    nodes[i].metadata["course"] = re.findall(course_code_pattern, nodes[i].text)

In [95]:
nodes[2].metadata

{'file_path': 'C:\\Users\\leduc\\OneDrive\\Desktop\\NLP\\LLM-Agent\\AI _solution\\sample-data\\text_2.txt',
 'file_name': 'text_2.txt',
 'file_type': 'text/plain',
 'file_size': 538,
 'creation_date': '2024-12-14',
 'last_modified_date': '2024-08-06',
 'excerpt_keywords': '"keywords: Môn Thực tập NT115, doanh nghiệp, kinh nghiệm"',
 'course': ['NT115']}

In [293]:
len(nodes)

13

In [96]:
splitted_documents = [Document(text=node.text, metadata={"course": node.metadata["course"], "excerpt_keywords": node.metadata["excerpt_keywords"]}) for node in nodes]

In [97]:
splitted_documents

[Document(id_='9ce860db-ab93-400c-97ba-7cd665db9ae0', embedding=None, metadata={'course': ['IE302'], 'excerpt_keywords': 'keywords: Kiến trúc hệ thống, Tích hợp hệ thống, IE302, Quản lý dự án, Quy trình tổ chức, Ứng dụng CNTT, Chất lượng, Tìm nguồn cung ứng.'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Môn học Kiến trúc hệ thống và Tích hợp hệ thống (IE302) giới thiệu các khái niệm về thu thập yêu cầu, tìm nguồn cung ứng, đánh giá và tích hợp các thành phần vào một hệ thống thống nhất. Nội dung môn học bao gồm các nguyên tắc quản lý dự án, sự liên kết giữa các ứng dụng CNTT và quy trình tổ chức. Sinh viên sẽ được trang bị kiến thức về quản lý dự án, kiểm định và đảm bảo chất lượng, tích hợp và phát triển hệ thống, cũng như tìm nguồn cung ứng.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 Doc

In [296]:
pinecone_index = pinecone.Index(host="https://agent-quoor8m.svc.aped-4627-b74a.pinecone.io", api_key= os.environ["PINECONE_API_KEY"])

In [297]:
storage_context = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(pinecone_index)
)

In [298]:
# index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
index = VectorStoreIndex.from_documents(splitted_documents, storage_context=storage_context, embed_model= embbeding_model)

Upserted vectors: 100%|██████████| 13/13 [00:03<00:00,  3.91it/s]


## Query transformation

In [32]:
from IPython.display import Markdown, display

In [33]:

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [61]:
example_query_prompt = """
You are a Vietnamese language expert. 

### Guidelines:
1. Response with Vietnamses
2. Only based on information about course information do not care about files information
3. Use natural and conversational Vietnamese suitable for the intended audience.
4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
5. remove all special token
6. you can not have an answer just reponse you don't know.
7. Do not Hallucinate if you don't know the answer.
8. Do not repeat the question.
9. If you see some mismatch between course name and course code you must return a notification

let's think step by step

response the question based on context:

question: {query_str}



context: {context_str}

answer:

"""

example_query_prompt = PromptTemplate(example_query_prompt)

In [35]:
query_engine = index.as_query_engine(llm=llm)

In [36]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [37]:
# query_engine.update_prompts(
#     {"response_synthesizer:refine_template": example_query_prompt},
# )

In [38]:

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": example_query_prompt},
)

In [39]:
query_str = "Môn CS115 cung cấp kiến thức gì cho người học ? "

In [176]:

response = query_engine.query(query_str)

In [177]:
response.response

'Các môn học có mã bắt đầu bằng chữ CS trong khoá học máy tính là CS115 (Toán ứng dụng) và CS116 (Lập trình C++). \n\n(Note: The context does not provide enough information to list all CS courses, only CS115 and CS116 are mentioned.)'

In [30]:
hyde_prompt = """
You are a Vietnamese language expert    

    1. Please write a passage to answer the question in Vietnamese
    2. Try to include as many key details as possible.
    3. Use natural and conversational Vietnamese suitable for the intended audience.
    4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
    5. remove all special token
    6. you can not have an answer just reponse you don't know.
    7. Do not Hallucinate if you don't know the answer.
        
    {context_str}
    
    
    Passage:
"""
    


In [31]:
hyde = HyDEQueryTransform(include_original=True, llm=llm, hyde_prompt=PromptTemplate(hyde_prompt))
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [32]:

response = hyde_query_engine.query(query_str)
response.response

'Môn học CE407 là môn học chuyên ngành Hệ thống nhúng và Robot dành cho sinh viên học chuyên đề tốt nghiệp. Trong môn này, sinh viên sẽ phát triển kỹ năng thiết kế hệ thống nhúng và robot. Sau khi hoàn thành môn học, sinh viên sẽ có kiến thức tổng hợp về chuyên ngành hệ thống nhúng và robot, kỹ năng tìm hiểu, nghiên cứu và giải quyết bài toán trong lĩnh vực hệ thống nhúng, kỹ thuật máy tính, cũng như kỹ năng thuyết trình, giao tiếp và làm việc'

## Query with Pinecone

In [8]:
import re
course_code_pattern = r"\b[A-Z]{2}[0-9]{3}\b"

In [9]:
index = pc.Index(host="https://agent-quoor8m.svc.aped-4627-b74a.pinecone.io")

In [10]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13}},
 'total_vector_count': 13}

In [11]:
def get_embed(text):
    return embbeding_model._embed(text)

In [31]:
query_text = "Tôi muốn tìm hiểu về môn toán cho khoa học máy tính IT005"
course_id = re.findall(course_code_pattern, query_text)
query_embedding = get_embed(query_text)

results = {}
if course_id:
    results = index.query(
        vector= query_embedding,
        filter={"course": {"$in": course_id}},
        top_k=3,
        include_metadata=True
    )
else:
    results = index.query(
        vector= query_embedding,
        top_k=5,
        include_metadata=True
    )


In [44]:
# results

In [33]:
import json

def result_content(results):
    matches = results['matches']  
    response_texts = []
    for match in matches:
        node_content = json.loads(match['metadata']['_node_content'])
        match_text = node_content['text']
        response_texts.append(match_text)
    response = "\n".join(response_texts)
    return response

print(result_content(results=results))


Môn Nhập môn mạng máy tính với mã IT005 cung cấp kiến thức cơ bản về các khái niệm như mạng máy tính, truyền dữ liệu, dịch vụ mạng và kỹ thuật mạng không dây.


## Hybrid search

In [8]:
Settings.llm = llm
Settings.embed_model = embbeding_model

In [9]:
reader = SimpleDirectoryReader(input_dir=r"C:\Users\leduc\OneDrive\Desktop\NLP\LLM-Agent\AI _solution\sample-data")
documents = reader.load_data(num_workers=4)

In [10]:
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=80, embed_model=embbeding_model
)


In [112]:
keyword_prompt = """
{context_str}. Give {keywords} unique Vietnamese keywords or course code for this \
document. Format as comma separated. let's think step by step. Keywords:
"""

In [113]:
transformations = [
    splitter,
    KeywordExtractor(keywords=1, llm=llm, prompt_template= keyword_prompt),
    # QuestionsAnsweredExtractor(questions=1, llm=llm, prompt_template= FAQ_prompt)
]

In [114]:
pipeline = IngestionPipeline(transformations=transformations)
nodes = pipeline.run(documents = documents)

100%|██████████| 10/10 [00:02<00:00,  3.81it/s]


In [122]:
import re

def clean_text(text):

    text = re.sub(r'\d+\.\s*', '', text)  
    

    text = re.sub(r'Keywords:.*', '', text)
    text = re.sub(r'-', ' ', text)
    text = text.replace('\n', ' ').strip()
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip()
    
    return text

In [123]:
for i in range(0, len(nodes)):
    print(clean_text(nodes[i].metadata["excerpt_keywords"]))
    nodes[i].metadata["excerpt_keywords"] = clean_text(nodes[i].metadata["excerpt_keywords"])


Quản lý dự án (Project management) Kiểm định và đảm bảo chất lượng (Quality assurance) Tích hợp và phát triển hệ thống (System integration and development) Quy trình tổ chức (Organizational process) CNTT (Information technology) Course code: IE302 (System Integration and Management)
Toán ứng dụng, CSThis document appears to be a text file containing information about a computer science course related to applied mathematics, specifically mathematics used in machine learning, artificial intelligence, data mining, and signal processing. The keywords or course code for this document could be "Toán ứng dụng" (Applied Mathematics) or "CS115" (Computer Science Course Code for Applied Mathematics).
Môn Thực tập (Practicum) Doanh nghiệp (Business) Chương trình thực tập cuối khóa (Capstone Practicum) Cơ quan sự nghiệp (Government agency) Lĩnh vực liên quan (Related field) Sinh viên (Student) Chuyên gia (Expert) Các công việc (Work tasks) Tích lũy kinh nghiệm (Gain experience) So the Vietnamese k

In [120]:
len(nodes)

13

In [149]:
splitted_documents = [Document(text=node.text, metadata={"excerpt_keywords": node.metadata["excerpt_keywords"]}) for node in nodes]

In [150]:
docstore = SimpleDocumentStore()
docstore.add_documents(splitted_documents)

storage_context = StorageContext.from_defaults(
    docstore=docstore
)


index = VectorStoreIndex(splitted_documents, storage_context=storage_context)

In [9]:
from llama_index.core import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir=r"C:\Users\leduc\OneDrive\Desktop\NLP\LLM-Agent\AI _solution\index")

In [10]:
index =  load_index_from_storage(storage_context)

In [119]:
# index.storage_context.persist("index/")


['2023']


In [16]:
import nest_asyncio

nest_asyncio.apply()

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=5),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=5
        ),
    ],
    # mode="reciprocal_rerank",
    num_queries=1,
    use_async=True,
    # query_gen_prompt = hyde_prompt
)

In [17]:
nodes = retriever.retrieve("Môn học Toán Cho Khoa Học Máy Tính CE407 là gì")
for node in nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 5eaa6195-107d-4b31-a308-78387bbd945a<br>**Similarity:** 4.021712303161621<br>**Text:** Mã môn học của môn Toán cho Khoa học máy tính là CS115. Tóm tắt nội dung môn học như sau: Môn này cung cấp kiến thức về Toán ứng dụng trong các lĩnh vực như máy học, trí tuệ nhân tạo, khai phá dữ liệu và xử lý tín hiệu số.<br>

**Node ID:** 0d29065d-d35e-44ae-b2dc-7420b773c245<br>**Similarity:** 3.1310784816741943<br>**Text:** Môn học CE407 - Đồ án chuyên ngành Hệ thống nhúng và Robot dành cho sinh viên học chuyên đề tốt nghiệp. Đồ án này giúp sinh viên phát triển kỹ năng thiết kế hệ thống nhúng và robot. Sau khi hoàn thành môn học, sinh viên sẽ có kiến thức tổng hợp về chuyên ngành hệ thống nhúng và robot, kỹ năng tìm hiểu, nghiên cứu và giải quyết bài toán trong lĩnh vực hệ thống nhúng, kỹ thuật máy tính, cũng như kỹ năng thuyết trình, giao tiếp và làm việc nhóm. Môn học cũng khuyến khích thái độ làm việc tích cực trong lĩnh vực hệ thống nhúng.<br>

In [18]:
cohere_rerank = CohereRerank(model="rerank-v3.5" ,api_key=os.environ["COHERE_API_TOKEN"], top_n=2)

c:\Python311\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [19]:
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[cohere_rerank])

In [33]:
chat_engine = index.as_chat_engine(chat_mode="context", query_engine = hyde_query_engine ,verbose=True, llm=llm)

In [34]:
query_str = "Môn học Toán Cho Khoa Học Máy Tính CE407 là gì"

In [35]:
react_response = chat_engine.chat(query_str)
print(react_response.response)

Đôi khi có lỗi phản ánh trong bạn đang yêu cầu về môn học Toán cho Khoa Học Máy Tính (Computer Science) với mã môn học CE407. Tuy nhiên, theo thông tin trong bài viết, CE407 là một môn học về Hệ thống nhúng và Robotics, không phải là môn học Toán cho Khoa Học Máy Tính. Nếu bạn đang tìm kiếm một môn học về Toán ứng dụng trong Khoa Học Máy Tính, bạn có thể xem xét mã môn học CS115 hoặc "Toán ứng dụng" làm từ khóa tìm kiếm.


In [25]:
example_query_prompt = """
You are a Vietnamese language expert. 

### Guidelines:
1. Response with Vietnamese
2. Only based on information about course information do not care about files information
3. Use natural and conversational Vietnamese suitable for the intended audience.
4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
5. remove all special token
6. you can not have an answer just reponse you don't know.
7. Do not Hallucinate if you don't know the answer.
8. Do not repeat the question.


let's think step by step

response the question based on context:

question: {query_str}

context: {context_str}

answer:

"""

example_query_prompt = PromptTemplate(example_query_prompt)

In [26]:

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": example_query_prompt},
)

In [168]:
response = query_engine.query(query_str)
print(response.response)

Môn học CE407 là một môn học chuyên ngành về hệ thống nhúng và robot dành cho sinh viên học chuyên đề tốt nghiệp. Trong môn này, sinh viên sẽ phát triển kỹ năng thiết kế hệ thống nhúng và robot, có kiến thức tổng hợp về chuyên ngành hệ thống nhúng và robot, kỹ năng tìm hiểu, nghiên cứu và giải quyết bài toán trong lĩnh vực hệ thống nhúng, kỹ thuật máy tính, cũng như kỹ năng thuyết trình, giao tiếp và làm việc nhóm.


In [24]:
hyde = HyDEQueryTransform(include_original=True, llm=llm, hyde_prompt=PromptTemplate(hyde_prompt))
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

NameError: name 'hyde_prompt' is not defined

In [188]:
query_str = "Môn học Toán Cho Khoa Học Máy Tính CE407 là gì"

In [172]:

hyde_response = hyde_query_engine.query(query_str)
print(hyde_response.response)

Môn học CE407 là một môn học chuyên ngành về hệ thống nhúng và robot dành cho sinh viên học chuyên đề tốt nghiệp. Trong môn này, sinh viên sẽ phát triển kỹ năng thiết kế hệ thống nhúng và robot, có kiến thức tổng hợp về chuyên ngành hệ thống nhúng và robot, kỹ năng tìm hiểu, nghiên cứu và giải quyết bài toán trong lĩnh vực hệ thống nhúng, kỹ thuật máy tính, cũng như kỹ năng thuyết trình, giao tiếp và làm việc nhóm.


In [173]:
pprint_response(hyde_response, show_source=True)

Final Response: Môn học CE407 là một môn học chuyên ngành về hệ thống
nhúng và robot dành cho sinh viên học chuyên đề tốt nghiệp. Trong môn
này, sinh viên sẽ phát triển kỹ năng thiết kế hệ thống nhúng và robot,
có kiến thức tổng hợp về chuyên ngành hệ thống nhúng và robot, kỹ năng
tìm hiểu, nghiên cứu và giải quyết bài toán trong lĩnh vực hệ thống
nhúng, kỹ thuật máy tính, cũng như kỹ năng thuyết trình, giao tiếp và
làm việc nhóm.
______________________________________________________________________
Source Node 1/2
Node ID: 5eaa6195-107d-4b31-a308-78387bbd945a
Similarity: 4.021712303161621
Text: Mã môn học của môn Toán cho Khoa học máy tính là CS115. Tóm tắt
nội dung môn học như sau: Môn này cung cấp kiến thức về Toán ứng dụng
trong các lĩnh vực như máy học, trí tuệ nhân tạo, khai phá dữ liệu và
xử lý tín hiệu số.
______________________________________________________________________
Source Node 2/2
Node ID: 0d29065d-d35e-44ae-b2dc-7420b773c245
Similarity: 3.1310784816741943
Text: Môn

In [5]:
pinecone_index = pinecone.Index(host="https://agent-quoor8m.svc.aped-4627-b74a.pinecone.io", api_key= os.environ["PINECONE_API_KEY"])
storage_context_pinecone = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(pinecone_index)
)
index = VectorStoreIndex.from_documents(splitted_documents, storage_context=storage_context_pinecone, embed_model= embbeding_model)

Upserted vectors: 100%|██████████| 439/439 [00:06<00:00, 68.55it/s]


In [57]:
import pandas as pd
df = pd.read_csv(r"C:\Users\leduc\OneDrive\Desktop\NLP\LLM-Agent\AI _solution\evalutation.csv", index_col=0)

In [58]:
df.head()

,reference,query,response,respone
0,Mã môn học của môn Toán cho Khoa học máy tính ...,Môn CS115 học cái gì?,NaN,NaN
1,Cử nhân ngành Công nghệ Thông tin (Áp dụng từ ...,Cử nhân ngành Công nghệ thông tin cần học mấy ...,NaN,NaN
2,Môn Nhập môn mạng máy tính với mã IT005 cung c...,Môn Nhập môn mạng máy tính học những gì?,NaN,NaN
3,Môn học CS221 về Xử lý ngôn ngữ tự nhiên nhằm ...,CS221 là môn gì?,NaN,NaN
4,Môn học mang tên Lý Luận Kinh Tế Chính Trị Má́...,Môn kinh tế chính trị Mác-Leenin là môn gì?,NaN,NaN


In [127]:
import pdfplumber
from llama_index.core import SimpleDirectoryReader, Document, VectorStoreIndex, StorageContext,  ChatPromptTemplate
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter, SentenceWindowNodeParser, SemanticSplitterNodeParser
# from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import Settings

from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)


from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.huggingface import HuggingFaceLLM, HuggingFaceInferenceAPI
from llama_index.llms.openai import OpenAI
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core import PromptTemplate



# from llama_index.core.postprocessor import LLMRerank, SentenceTransformerRerank
# from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core import load_index_from_storage


import openai
from pinecone import Pinecone
import pinecone
import os
from dotenv import load_dotenv
import json


import asyncio
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

pc = Pinecone(api_key = os.environ["PINECONE_API_KEY"])
openai.api_key = os.environ["OPEN_AI_KEY"]
embbeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2", token=os.environ["HF_TOKEN"])
Settings.llm = llm
Settings.embed_model = embbeding_model

import re
course_code_pattern = r"\b[A-Z]{2}[0-9]{3}\b"

example_query_prompt = """
You are a Vietnamese language expert. 

### Guidelines:
1. Response with Vietnamese
2. Only based on information about course information do not care about files information
3. Use natural and conversational Vietnamese suitable for the intended audience.
4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
5. remove all special token
6. you can not have an answer just reponse you don't know.
7. Do not Hallucinate if you don't know the answer.
8. Do not repeat the question.


let's think step by step

response the question based on context:

question: {query_str}

context: {context_str}

answer:

"""

hyde_prompt = """
You are a Vietnamese language expert    

    1. Please write a passage to answer the question in Vietnamese
    2. Try to include as many key details as possible.
    3. Use natural and conversational Vietnamese suitable for the intended audience.
    4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
    5. remove all special token
    6. you can not have an answer just reponse you don't know.
    7. Do not Hallucinate if you don't know the answer.
    8. Only use Vietnamese for answer
        
    {context_str}
    
    
    Passage:
"""
    

def extract_years(text):
    pattern = r'20[0-9][0-9]'
    return re.findall(pattern, text)

# Load local index
storage_context = StorageContext.from_defaults(persist_dir=r"./rag_database")
index =  load_index_from_storage(storage_context)


# implement hybrid search, keywords search, metadata filtering, HyDE query transformation
retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=10),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=10
        ),
    ],
    num_queries=1,
    use_async=True,
)

cohere_rerank = CohereRerank(model="rerank-v3.5" ,api_key=os.environ["COHERE_API_TOKEN"], top_n=4)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[cohere_rerank])


example_query_prompt = PromptTemplate(example_query_prompt)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": example_query_prompt},
)

hyde = HyDEQueryTransform(include_original=True, llm=llm, hyde_prompt=PromptTemplate(hyde_prompt))
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

chat_engine = index.as_chat_engine(chat_mode="condense_plus_context", system_prompt=(chat_prompt) ,query_engine = query_engine ,verbose=True, llm=llm, max_tokens = 1000)




# Ham xuat ket qua cho Pinecone
def result_content(results):
    matches = results['matches']  
    response_texts = []
    for match in matches:
        node_content = json.loads(match['metadata']['_node_content'])
        match_text = node_content['text']
        response_texts.append(match_text)
    response = "\n".join(response_texts)
    return response

# connect voi Pinecone Index
pinecone_index = pc.Index(host="https://agent-quoor8m.svc.aped-4627-b74a.pinecone.io")

def get_embed(text):
    return embbeding_model._embed(text)


def get_chat(query_str):
    """
    Params:
    year -> list: contain extracted years from query
    query_embedding: tu biet di
    query_str -> str: tu biet di
    
    Output:
    str: response
    """
    query_embedding = get_embed(query_str)
    year = extract_years(query_str)
   
    if year:
        results = pinecone_index.query(
            vector= query_embedding,
            filter={"year": {"$in": year}},
            top_k=5,
            include_metadata=True
        )
        res = result_content(results)
        react_response = chat_engine.chat(query_str + "\n" + res)
        chat_engine.reset()
        return react_response.response
    else:
        react_response = chat_engine.chat(query_str)
        chat_engine.reset()
        return react_response.response
    


C:\Users\leduc\AppData\Local\Temp\ipykernel_17600\2595136526.py:56: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2", token=os.environ["HF_TOKEN"])


In [82]:
# res = []
# idx = 0
# for query in df["query"]:
#     print(idx)
#     res.append(get_chat(query))
#     chat_engine.reset()
#     idx+=1

In [74]:
# for i in range(len(res)):
#     res[i] = res[i].response


In [68]:
# df["response"] = res

In [102]:
# res

In [ ]:
# df.to_csv(r"C:\Users\leduc\OneDrive\Desktop\NLP\LLM-Agent\AI _solution\results.csv")

In [128]:
query_str = "tín chỉ tốt nghiệp ngành khoa học máy tính năm 2023 là bao nhiêu ?"

In [126]:
chat_prompt = """
You are a Vietnamese language expert. 

### Guidelines:
1. Response with Vietnamese
2. Only based on information about course information do not care about files information
3. Use natural and conversational Vietnamese suitable for the intended audience.
4. Avoid repetition and ensure each question addresses a distinct aspect of the context.
5. remove all special token
6. Short answer
7. Do not Hallucinate if you don't know the answer.
8. Do not repeat the question.


let's think step by step

response the question based on context:


"""

In [122]:
# chat_engine = index.as_chat_engine(chat_mode="condense_plus_context", query_engine = query_engine ,verbose=True, llm=llm)
# chat_engine.reset()

In [123]:
# df.head(20)

In [129]:
ans = get_chat(query_str=query_str)


Condensed question: tín chỉ tốt nghiệp ngành khoa học máy tính năm 2023 là bao nhiêu ?
Cử nhân ngành Khoa học Máy tính (Áp dụng từ khóa 18 - 2023) Hình thức đào tạo: Chính quy tập trung. Số tín chỉ đào tạo: 126 tín chỉ Cấp bằng: Cử nhân ngành Khoa học Máy tính. Thời gian đào tạo: 3.5 năm (gồm 7 học kỳ chính thức).
Cử nhân ngành Khoa học Máy tính (Áp dụng từ khóa 18 - 2023) Mục tiêu chung: Chương trình đào tạo hướng đến đào tạo nguồn nhân lực Khoa học máy tính chất lượng cao đạt trình độ khu vực và quốc tế, đáp ứng yêu cầu công nghiệp hóa, hiện đại hóa đất nước và hội nhập quốc tế. Trang bị cho người học kiến thức cơ bản và chuyên sâu về Khoa học máy tính và công nghệ thông tin. Người học có những kiến thức, kỹ năng và thái độ chuẩn mực trong phát triển nghề nghiệp.
Cử nhân ngành Kỹ thuật Máy tính (Áp dụng từ khóa 18 - 2023) Hình thức đào tạo: Chính quy tập trung.- Số tín chỉ đào tạo: 128 tín chỉ. - Thời gian đào tạo: 4 năm (8 học kỳ)
Cử nhân ngành Công nghệ Thông tin (Áp dụng từ Kho

In [130]:

print(ans)

Đối với năm 2023, tín chỉ tốt nghiệp cho ngành Khoa học Máy tính là 126 tín chỉ.


1
